In [ ]:
import matplotlib
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import sys
import matplotlib.pyplot as plt
from sklearn import svm
from jedi.api.refactoring import inline

from sklearn.datasets import load_boston
from sklearn.datasets import load_iris, make_moons
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, accuracy_score, classification_report
from sklearn.pipeline import make_pipeline, Pipeline
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, cross_val_predict, train_test_split
from sklearn.base import clone
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn import tree
from sklearn.tree import export_graphviz
import graphviz
from sklearn import preprocessing

# PROYECTO HAGAR DE IBM: Predecir el desgaste de sus empleados disponibles

df = pd.read_csv("C:/Users/Armando/Documents/Ciencia_Datos/Udemy/Datos/IBM/WA_Fn-UseC_-HR-Employee-Attrition.csv")
print(df.head())

# EDA
print(df.shape)             # Indica dimensionalidad de los datos (1470 filas, 35 columnas)
print(df.info())            # Muestra el tipo de dato de cada columna

print(df.iloc[0])           # iloc selecciona filas de acuerdo con el número que se indique. En este caso se selecciona la primera fila

print(df.isnull().any())    #Muestra si existen nulos en nuestro conjunto de datos

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#print(df.describe())        #Estadística medidas de tendencia central (media, desviación standard, mediana, cuartiles, etc)

num_col = list(df.describe().columns)                       # Inserta en listado el nombre de las columnas
col_categorical = list(set(df.columns).difference(num_col))         # Indica las columnas que son categóricas de las 35 columnas en total
remove_list = ['EmployeeCount', 'EmployeeNumber', 'StandardHours']
col_numerical = [e for e in num_col if e not in remove_list]

#print(num_col)
#print(col_categorical)
print(col_numerical)

categorical_col = []

for k, v in df.iteritems():
    if (v.dtype == 'object'):
        categorical_col.append(k)

ColumnasCategoricas = 'Columnas categóricas: {}'.format(str(len(categorical_col)))
ColumnasNumericas = 'Columnas numéricas: {}'.format(str(len(col_numerical)))

print('Columnas:' + str(len(num_col)))
print(ColumnasCategoricas)
print(ColumnasNumericas)


ColNumericalCorr = df[col_numerical].corr()
print(ColNumericalCorr)

plt.figure(figsize=(24, 8))
sns.heatmap(df[col_numerical].corr(), annot=True, fmt='.2f')
plt.show()

plt.figure(figsize=(24, 8))
mask = np.zeros_like(df[col_numerical].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

#Genera un mapa de colores divergente
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Mapa de Calor con ratio de aspectos correctos
sns.heatmap(df[col_numerical].corr(), mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()


plt.figure(figsize=(24, 8))

# Mascara para el triangulo superior
mask = np.zeros_like(df[col_numerical].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.light_palette((210, 90, 60), input='husl')

# Mapa de calor con mascara
sns.heatmap(df[col_numerical].corr(), mask=mask, cmap=cmap, annot=True, fmt='.2f')
plt.show()

df['Attrition'].unique()

attrition_to_num = {'Yes': 0, 'No': 1}
df['Attrition_num'] = df['Attrition'].map(attrition_to_num)

col_categorical.remove('Attrition')

df_cat = pd.get_dummies(df[col_categorical])
print(df_cat.head())

X = pd.concat([df[col_numerical], df_cat], axis=1)
print(X.head())

y = df['Attrition_num']
print(y.head())

print(y.value_counts())

print(X.info())


#Modelo con datos estandarizados

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

accuracy_score(y_train, clf.predict(X_train))
print(classification_report(y_train, clf.predict(X_train)))
confusion_matrix(y_train, clf.predict(X_train))

accuracy_score(y_test, clf.predict(X_test))
print(classification_report(y_test, clf.predict(X_test)))
confusion_matrix(y_test, clf.predict(X_test))

def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)

    if train:
        '''
        training process
        '''
        res = clf.predict(X_train)
        print("Train Result: \n")
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_train, res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, res)))
        print("Confussion Matrix: \n {}\n".format(confusion_matrix(y_train, res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), lb.transform(res))))

    elif train == False:

        res_test = clf.predict(X_test)
        print("Train Result: \n")
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_test, res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, res_test)))
        print("Confussion Matrix: \n {}\n".format(confusion_matrix(y_test, res_test)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))

print_score(clf, X_train, X_test, y_train, y_test, train=True)
print_score(clf, X_train, X_test, y_train, y_test, train=False)

